In [25]:
import numpy as np
import json
from bottle import Bottle, request, response, run, template, static_file

In [ ]:
#COPIED FROM MILESTONE 1 SUBMISSION
from pynq import Overlay, DefaultIP, allocate

class ConstantMultiplyDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)
        self._constant_r_addr = self.register_map.constant_r.address
    
    bindto = ['xilinx.com:hls:mult_constant:1.0']
    
    @property
    def constant(self):
        return self.read(self._constant_r_addr)
    @constant.setter
    def constant(self, value):
        self.write(self._constant_r_addr, value)
        
        
#TODO: UPDATE TO POINT TO CORRECT FILE AND SEND CORRECT DATA
overlay = Overlay('/home/xilinx/pynq/overlays/project_mult_constant/project_mult_constant.bit')
multiply = overlay.mult_constant_0
dma = overlay.axi_dma_0
in_buffer = allocate(shape=(5,), dtype=np.uint32)
out_buffer = allocate(shape=(5,), dtype=np.uint32)

In [29]:
app = Bottle()
strings = {}  # Store the values
next_id = 1  # Unique ID for each request

# Route to serve static CSS files
@app.route('/static/<filename>')
def serve_static(filename):
    return static_file(filename, root='./static')

# Route to render the HTML page
@app.get('/')
def index():
    return template("index", strings=strings)

#TODO: PUT HTML AND CSS FILES IN CORRECT PLACES

In [30]:
@app.post('/encrypt')
def encrypt():
    global next_id

    # Get the number value from the form
    input_value = request.forms.get('input')

    # Make sure a valid number is provided
    if not input_value:
        return "<p>Error: Text input is required.</p><a href='/'>Go Back</a>"
    
    try:
        input_value = str(input_value)  # Convert to integer
    except ValueError:
        return "<p>Error: Please enter valid text.</p><a href='/'>Go Back</a>"
 
    
    
    input_encrypt = aes_encrypt(input_value, key)
      
    
    #COPIED FROM MILESTONE 1 SUBMISSION
    # SEND num_value to board and do x2 on the FPGA, then send value back here
   
    #TODO: REPLACE NUMBERS WITH KEY AND INPUT VAL
    key = "thisisakey123456"
    in_buffer[0] = input_value
    in_buffer[1] = key
    in_buffer

    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()

    input_encrypt = out_buffer[0]
   
    # Store the request and its result
    input_id = next_id
    next_id += 1
    strings[input_id] = {"id": input_id, "value": input_value, "encrypt": input_encrypt}

    
    
    # Show the success message and return to the main page
    return f"<p>Input {input_value} \nEncryption : {input_encrypt}</p><a href='/'>Go Back</a>"


In [31]:
# Run the app
if __name__ == '__main__':
    run(app, host='192.168.2.99', port=8080, debug=True)

Bottle v0.13.2 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [19/Feb/2025 14:59:17] "GET / HTTP/1.1" 200 826
127.0.0.1 - - [19/Feb/2025 14:59:17] "GET /static/style.css HTTP/1.1" 304 0
127.0.0.1 - - [19/Feb/2025 14:59:21] "POST /encrypt HTTP/1.1" 200 89
127.0.0.1 - - [19/Feb/2025 14:59:24] "GET / HTTP/1.1" 200 904
